# LOG6302A — Analyse d’applications et Cyber-sécurité<br>Laboratoire #6

**Quentin Guidée (2206809), Nam Vu (2230468)**

Polytechnique Montréal – Hiver 2024


## Imports et helpers

In [62]:
import tempfile
from collections import defaultdict
from itertools import product
from pathlib import Path

import numpy as np
from graphviz import Source
from IPython.display import Image

from code_analysis import AST, AST_fragmentation, ASTReader, Graph

In [31]:
def show_graph(graph: Graph):
    """Afficher le graphe dans Jupyter"""
    dot = graph.to_dot()
    s = Source(dot)
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        s.render(f.name, format="png")
        display(Image(f.name + ".png"))

## Prise en main

Nous parsons une fois pour toute les AST de tous les fichiers de tous les kits :

In [32]:
reader = ASTReader()
path_ast = Path("ast")


def get_kits():
    return [dir_kit for dir_kit in path_ast.iterdir() if dir_kit.is_dir()]


def get_kit_asts(kit: Path):
    return {file: reader.read_ast(file.as_posix()) for file in kit.glob("**/*.ast.*")}


all_kits_asts = {kit: get_kit_asts(kit) for kit in get_kits()}

## Partie 1 : Fichiers

### Clones paramétriques

Dans cette section nous cherchons tous les fichiers qui ont des vecteurs strictement identiques entre eux.

In [33]:
def parametric_file_groups(all_kits_asts: dict[Path, dict[Path, AST]]):
    vector_groups = defaultdict[tuple[int, ...], list[Path]](list)

    for kit in all_kits_asts.values():
        for file_path, ast in kit.items():
            if len(ast.get_node_ids()) >= 100:
                vector_groups[tuple(ast.vectorize())].append(file_path)

    return vector_groups

In [34]:
groups = parametric_file_groups(all_kits_asts)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(47,
 [PosixPath('ast/4417/rabo2020/def/pin/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/def/info/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/start/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/info/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/cc/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/start/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/info/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/cc/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/start/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/info/index.php.ast.json.gz'),
  PosixPath('ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/cc/index.php.ast.json.gz'),
  Pos

Dans le kit 2099 on retrouve 3 fichiers qui ont des vecteurs strictement identiques entre eux, copiés à plusieurs reprises dans plusieurs sous dossiers : `start/index.php`, `info/index.php` et `cc/index.php`.

Ces fichiers sont aussi identiques aux deux fichiers `def/pin/index.php` et `def/info/index.php` du kit 4417.

On refait donc la recherche en ignorant les fichiers dupliqués dans un même kit :

In [35]:
def parametric_file_groups_deduped(all_kits_asts: dict[Path, dict[Path, AST]]):
    vector_groups = defaultdict[tuple[int, ...], list[Path]](list)

    for kit in all_kits_asts.values():
        dedup = set[tuple[int, ...]]()
        for file_path, ast in kit.items():
            if len(ast.get_node_ids()) >= 100:
                vec = tuple(ast.vectorize())
                if vec not in dedup:
                    vector_groups[vec].append(file_path)
                    dedup.add(vec)

    return vector_groups

In [36]:
groups = parametric_file_groups_deduped(all_kits_asts)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(7,
 [PosixPath('ast/3135/hunting/antibot/anti8.php.ast.json.gz'),
  PosixPath('ast/1442/web/auth/Bots/anti8.php.ast.json.gz'),
  PosixPath('ast/1110/submit/prevents/anti8.php.ast.json.gz'),
  PosixPath('ast/0485/we logz sure/anti/anti8.php.ast.json.gz'),
  PosixPath('ast/0415/M&T/darkx/anti8.php.ast.json.gz'),
  PosixPath('ast/3662/usps-shippment/bots/anti8.php.ast.json.gz'),
  PosixPath('ast/3716/gruposantander/xJOESTAR/anti8.php.ast.json.gz')])

On observe que l'on retrouve le même fichier `anti8.php` à travers 7 kits différents.

Au vu des chemins d'accès il semble qu'il s'agisse d'un fichier permettant de bloquer les robots et donc de réduire les chances de détection de ces kits (cloaking).

### Fichiers simillaires

Dans cette section, on identifie les fichiers simillaires à l'aide de la distance de Manhattan.

On commence par vectoriser les AST des fichiers de tous les kits :

In [37]:
def vectorize_kit_asts(all_kits_asts: dict[Path, dict[Path, AST]]):
    ast_vectors: dict[Path, np.ndarray] = {}

    for kit in all_kits_asts.values():
        for file_path, ast in kit.items():
            if len(ast.get_node_ids()) >= 100:
                ast_vectors[file_path] = ast.vectorize()

    return ast_vectors

In [38]:
ast_vectors = vectorize_kit_asts(all_kits_asts)
ast_vectors

{PosixPath('ast/2620/absa/php/continue3.php.ast.json.gz'): array([ 0,  0,  0,  0, 22, 24,  0,  0,  5, 15, 55,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  0,  0,  0,  0,  0,
        18,  0,  0,  3,  0,  0,  1,  1,  0,  0,  0,  0,  2,  0,  0,  0,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  1,  6,  0,  0, 23, 33,  0,  0,  0,  0,  0,  0,  0,  2,  0,
         0,  0,  0,  0, 70,  0,  0,  0]),
 PosixPath('ast/2620/absa/php/continue4.php.ast.json.gz'): array([ 0,  0,  0,  0, 25, 35,  0,  0,  5, 14, 56,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,
        21,  0,  0,  3,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,
      

La méthode suivante permet de grouper les vecteurs similaires :

In [39]:
def similar_groups[T](ast_vectors: dict[T, np.ndarray], threshold: float):
    similarities = defaultdict[T, list[T]](list)

    for (i, vector_i), (j, vector_j) in product(ast_vectors.items(), repeat=2):
        if i == j:
            continue
        if np.abs(vector_j - vector_i).sum() <= threshold * np.sum(vector_i):
            similarities[i].append(j)

    return similarities

On l'applique à nos AST :

In [40]:
groups = similar_groups(ast_vectors, 0.3)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(80,
 [PosixPath('ast/2110/moneylion/def/token/index - Copy.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/wifi/index.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/read/index.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/token2/index.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/token3/index.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/cc/index.php.ast.json.gz'),
  PosixPath('ast/2110/moneylion/def/login/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/token/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/done/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/login/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/a1b2c3/44a24055e71e417ccebb479ccef7bacf/token/index.php.ast.json.gz'),
  PosixPath('ast/4417/rabo2020/a1b2c3/44a24055e71e417ccebb479ccef7bacf/done/index.php.ast.json.gz'),
  PosixPath('a

On retrouve le premier cluster détecté à la section précédente. On observe néanmoins que ces fichiers se retrouvent aussi légèrement modifiés dans le reste du kit 4417 ainsi que dans le kit 2110.

Il est d'ailleurs intéressant de noter que dans le kit 2110 un des fichiers similaire porte le nom de `index - Copy.php`...

## Partie 2 : Fragments

Dans cette partie on s'intéresse la similarité des fragments qui composent les AST des différents kits.

### Clones paramétriques

In [41]:
def parametric_fragment_groups(all_kits_asts: dict[Path, dict[Path, AST]]):
    vector_groups = defaultdict[tuple[int, ...], list[tuple[Path, str]]](list)

    for kit in all_kits_asts.values():
        dedup = set[tuple[int, ...]]()
        for file_path, ast in kit.items():
            fragments: list[int] = AST_fragmentation(ast)
            fragments.append(ast.get_root())  # prendre en compte le root aussi
            for fragment in fragments:
                if len(ast.dfs(fragment)) >= 10:
                    vec = tuple(ast.vectorize(fragment))
                    if vec not in dedup:
                        vector_groups[vec].append((file_path, ast.get_image(fragment)))
                        dedup.add(vec)

    return vector_groups

In [42]:
groups = parametric_fragment_groups(all_kits_asts)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(12,
 [(PosixPath('ast/2070/AdobeDoc/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/2110/moneylion/index.php.ast.json.gz'), 'dublicate'),
  (PosixPath('ast/4417/rabo2020/index.php.ast.json.gz'), 'dublicate'),
  (PosixPath('ast/3180/secure/login/dir.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/2099/dhl/index.php.ast.json.gz'), 'dublicate'),
  (PosixPath('ast/1651/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/3676/chase-online/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/1110/submit/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/0009/CA 3.0/CA 3.0/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/0229/io/Login/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/4218/outlook_owa/index.php.ast.json.gz'), 'recurse_copy'),
  (PosixPath('ast/3548/owa/index.php.ast.json.gz'), 'recurse_copy')])

Le plus gros cluster est composé de 12 fragments strictement identiques. Ces méthodes portent les noms de `recurse_copy` et `dublicate` (la même typo est présente dans les différents kits).

### Fragements similaires

On commence par vectoriser les fragments :

In [43]:
def vectorize_fragment_asts(all_kits_asts: dict[Path, dict[Path, AST]]):
    ast_vectors: dict[tuple[Path, str], np.ndarray] = {}

    for kit in all_kits_asts.values():
        dedup = set[tuple[int, ...]]()
        for file_path, ast in kit.items():
            fragments: list[int] = AST_fragmentation(ast)
            fragments.append(ast.get_root())  # prendre en compte le root aussi
            for fragment in fragments:
                if len(ast.dfs(fragment)) >= 10:
                    vec = ast.vectorize(fragment)
                    if tuple(vec) not in dedup:
                        ast_vectors[(file_path, ast.get_image(fragment))] = vec
                        dedup.add(tuple(vec))

    return ast_vectors

In [44]:
fragment_vectors = vectorize_fragment_asts(all_kits_asts)
fragment_vectors

{(PosixPath('ast/2620/absa/images/logo-blu.png.ast.json.gz'),
  None): array([0, 0, 0, 0, 3, 8, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 3, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0]),
 (PosixPath('ast/2620/absa/images/js.php.ast.json.gz'),
  None): array([0, 0, 0, 0, 4, 6, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 4, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 6, 4,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]),
 (Po

On réutilise la méthode `similar_groups` de la partie 1 :

In [45]:
groups = similar_groups(fragment_vectors, 0.1)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(15,
 [(PosixPath('ast/1926/CITIZ/dead.php.ast.json.gz'), 'getOS'),
  (PosixPath('ast/0481/mazon/amazon/XBALTI/get_browser.php.ast.json.gz'),
   'XB_OS'),
  (PosixPath('ast/3858/onlineaqs.wellsfargo.us/Spox/Functions/Fuck-you.php.ast.json.gz'),
   'getOs'),
  (PosixPath('ast/0109/cappy/s/Bots/bot/Antibot/Module/Setmodule.php.ast.json.gz'),
   'getOS'),
  (PosixPath('ast/1442/web/auth/XBALTI/send.php.ast.json.gz'), 'XB_OS'),
  (PosixPath('ast/2866/mazon/amazon/XBALTI/get_browser.php.ast.json.gz'),
   'XB_OS'),
  (PosixPath('ast/1651/inc/functions.php.ast.json.gz'), 'get_user_os'),
  (PosixPath('ast/1110/submit/inc/functions.php.ast.json.gz'), 'get_user_os'),
  (PosixPath('ast/0406/VYSTARBANK[MRWEEBEE]/dead.php.ast.json.gz'), 'getOS'),
  (PosixPath('ast/0431/includes/userinfo.php.ast.json.gz'), 'getOS'),
  (PosixPath('ast/0229/io/Login/inc/functions.php.ast.json.gz'),
   'get_user_os'),
  (PosixPath('ast/0415/M&T/darkx/recon.php.ast.json.gz'), 'getOs'),
  (PosixPath('ast/4273/mazon/amazo

Le plus gros cluster est composé de 15 fragments similaires. Les méthodes portent le nom de `getOS`, `XB_OS` ou encore `get_user_os`. Elles servent probablement à récupérer le système d'exploitation de l'utilisateur.

## Partie 3 : Kits paramétriques

### Détection des kits identiques

Commençons par sommer les vecteurs des AST à l'intérieur de chaque kit :

In [46]:
def kit_sums(all_kits_asts: dict[Path, dict[Path, AST]]):
    return {
        p: np.sum([ast.vectorize() for ast in kit.values()], axis=0)
        for p, kit in all_kits_asts.items()
    }

In [47]:
all_sums = kit_sums(all_kits_asts)
all_sums

{PosixPath('ast/2620'): array([   0,    0,    0,    0, 2939, 3574,    7,    0,  253, 1741, 4503,
         617,    0,    0,   54,   60,    6,    4,    1,    0,   16,    6,
           6,    0,  818,  103,  103,  103,    0,   12,    0,    4,    0,
           1,    0, 1199,    0,   43,   32,    0,    1,  145,    1,   69,
          40, 1130,   13,   12,    0,    0,   26, 2293,   20,   12,  229,
         327,    0,    5,    6,   69,   69,    0,    0,  412,    0,    0,
           0,  288,    0,  585,    3,    0,    0,    0,   81,    0,    0,
           0,    5,   27,   85,    1,    0,   81,    0,    0,    0,    6,
           5,    0,    6,    0,    0,    1,   64,    0,  614,    0,    0,
         139,    0,    0,   13,   72,   14, 1004,    0,    1, 3208, 2426,
           6,    0,    0,   71,    1,    0,    0,  131,   25,    0,    0,
           0,   74, 6004,    0,   12,   15]),
 PosixPath('ast/3135'): array([    0,     0,     0,     0,   745,  1121,     0,     0,   100,
          178,   553,  

Groupons les kits identiques :

In [48]:
def kit_groups(all_sums: dict[Path, np.ndarray]):
    vector_groups = defaultdict[tuple[int, ...], list[Path]](list)
    for p, ksum in all_sums.items():
        vector_groups[tuple(ksum)].append(p)
    return vector_groups

In [50]:
groups = kit_groups(all_sums)
groups = list(groups.values())
groups.sort(key=len, reverse=True)
cluster = groups[0]
len(cluster), cluster

(3, [PosixPath('ast/0481'), PosixPath('ast/2866'), PosixPath('ast/4273')])

On retrouve 3 kits identiques : 0481, 2866 ainsi que 4273.

### Comparaison des kits

On simplifie d'abord les données pour préparer la comparaison :

In [119]:
kits = {}
for path, kit in all_kits_asts.items():
    if path.name in ['0481', '2866', '4273']:
        for file_path, ast in kit.items():
            if str(path.name) not in kits:
                kits[str(path.name)] = {}
            kits[str(path.name)][str(file_path)[9:]] = ast
kits

{'0481': {'mazon/index.php.ast.json.gz': <code_analysis.AST.AST at 0x323494e90>,
  'mazon/amazon/antibots.php.ast.json.gz': <code_analysis.AST.AST at 0x3234b0260>,
  'mazon/amazon/index.php.ast.json.gz': <code_analysis.AST.AST at 0x3234b01d0>,
  'mazon/amazon/signin.php.ast.json.gz': <code_analysis.AST.AST at 0x3234b39b0>,
  'mazon/admin/rezulta.php.ast.json.gz': <code_analysis.AST.AST at 0x323497ef0>,
  'mazon/amazon/homepage/Card.php.ast.json.gz': <code_analysis.AST.AST at 0x3235c89e0>,
  'mazon/amazon/homepage/success.php.ast.json.gz': <code_analysis.AST.AST at 0x3235cb4d0>,
  'mazon/amazon/homepage/index.php.ast.json.gz': <code_analysis.AST.AST at 0x3235cb470>,
  'mazon/amazon/homepage/secure.php.ast.json.gz': <code_analysis.AST.AST at 0x3235e4920>,
  'mazon/amazon/homepage/email.php.ast.json.gz': <code_analysis.AST.AST at 0x3235e59d0>,
  'mazon/amazon/XBALTI/send_vbv.php.ast.json.gz': <code_analysis.AST.AST at 0x3235e7890>,
  'mazon/amazon/XBALTI/get_pass.php.ast.json.gz': <code_a

In [124]:
all_paths = set(path for kit in kits.values() for path in kit.keys())
all_paths

{'mazon/admin/rezulta.php.ast.json.gz',
 'mazon/amazon/XBALTI/Email.php.ast.json.gz',
 'mazon/amazon/XBALTI/check_bin.php.ast.json.gz',
 'mazon/amazon/XBALTI/get_browser.php.ast.json.gz',
 'mazon/amazon/XBALTI/get_ip.php.ast.json.gz',
 'mazon/amazon/XBALTI/get_pass.php.ast.json.gz',
 'mazon/amazon/XBALTI/send_billing.php.ast.json.gz',
 'mazon/amazon/XBALTI/send_card.php.ast.json.gz',
 'mazon/amazon/XBALTI/send_email.php.ast.json.gz',
 'mazon/amazon/XBALTI/send_login.php.ast.json.gz',
 'mazon/amazon/XBALTI/send_vbv.php.ast.json.gz',
 'mazon/amazon/antibots.php.ast.json.gz',
 'mazon/amazon/homepage/Card.php.ast.json.gz',
 'mazon/amazon/homepage/email.php.ast.json.gz',
 'mazon/amazon/homepage/index.php.ast.json.gz',
 'mazon/amazon/homepage/secure.php.ast.json.gz',
 'mazon/amazon/homepage/success.php.ast.json.gz',
 'mazon/amazon/index.php.ast.json.gz',
 'mazon/amazon/signin.php.ast.json.gz',
 'mazon/amazon/style/css/index.php.ast.json.gz',
 'mazon/amazon/style/img/index.php.ast.json.gz',
 

Nous parcourons ensuite tous les AST en parallèle pour trouver des différences :

In [170]:
def deep_compare(asts: list[AST], nids: list[int]):
    node_types = [ast.get_type(nid) for ast, nid in zip(asts, nids)]
    node_images = [ast.get_image(nid) for ast, nid in zip(asts, nids)]

    if len(set(node_types)) > 1:
        print("  Different node types:", node_types)
    
    if len(set(node_images)) > 1:
        print("  Different node images", node_images)

    children = [ast.get_children(nid) for ast, nid in zip(asts, nids)]
    for i in range(len(children[0])):
        deep_compare(asts, [child[i] for child in children])

for path in all_paths:
    asts = [kit[path] for kit in kits.values() if path in kit]
    print(f"Comparing {path}")
    deep_compare(asts, [ast.get_root() for ast in asts])

Comparing mazon/amazon/XBALTI/check_bin.php.ast.json.gz
Comparing mazon/amazon/XBALTI/send_vbv.php.ast.json.gz
Comparing mazon/amazon/homepage/success.php.ast.json.gz
Comparing mazon/admin/rezulta.php.ast.json.gz
Comparing mazon/amazon/signin.php.ast.json.gz
Comparing mazon/amazon/homepage/email.php.ast.json.gz
Comparing mazon/amazon/homepage/secure.php.ast.json.gz
Comparing mazon/amazon/XBALTI/get_pass.php.ast.json.gz
Comparing mazon/amazon/XBALTI/Email.php.ast.json.gz
  Different node images ['mr.alexandatony@gmail.com', 'collinshofmann@outlook.com, castilloalphonso@gmail.com', 'collinshofmann@outlook.com, castilloalphonso@gmail.com']
  Different node images ['Romay', 'Reward', 'Reward']
Comparing mazon/amazon/XBALTI/send_billing.php.ast.json.gz
Comparing mazon/amazon/XBALTI/send_card.php.ast.json.gz
Comparing mazon/amazon/style/img/index.php.ast.json.gz
Comparing mazon/amazon/antibots.php.ast.json.gz
Comparing mazon/amazon/index.php.ast.json.gz
Comparing mazon/amazon/XBALTI/get_ip.p

On remarque que les seules différences sont dans le kit 0481, dans le fichier `Email.php`. Les **types de nodes sont identiques**. En revanche, les **images diffèrent** des deux autres kits :
- L'email est `mr.alexandatony@gmail.com` au lieu de `collinshofmann@outlook.com, castilloalphonso@gmail.com` dans les deux autres
- Une autre image diffère aussi dans le kit 0481 : la chaine de caractères `Romay` au lieu de `Reward`.

Tout le reste semble identique du point de vue de l'AST.